In [ ]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import os
import time     # this library provides various time related functions like current time, date etc.

In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Read the initial data and have a look at the initial header to see wether you have unnmaed columns or rows
data_all_deaths=pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/covid_deaths_usafacts.csv")
data_all_cases=pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/All_states_daily_cases.csv")
data_all_deaths.head(5)# have look at the firt five rows

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,28,28,28,28,28,28,28,28,28,28,29,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,32,32,36,36,36,36,37,37
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,65,65,65,65,66,66,67,67,67,67,69,69,69,69,69,69,69,69,69,69,71,71,71,71,74,77,77,83,83,83,83,84,84,84,84,84,84,84,84,84
3,1005,Barbour County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,10
4,1007,Bibb County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,12,12,12,12,13,13,13,13,13,13,14,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15,15,16,16,16,17,17,17,17,17,17,17,17,18


In [ ]:
#Filterting the states (CA)
data_CA_deaths = data_all_deaths[data_all_deaths['State'].values=='CA']
data_CA_deaths.drop(data_CA_deaths.loc[data_CA_deaths['County Name']=="Statewide Unallocated"].index, inplace=True)
data_CA_deaths.drop(data_CA_deaths.loc[data_CA_deaths['County Name']=="Grand Princess Cruise Ship"].index, inplace=True)
data_CA_deaths = data_CA_deaths.sort_values(by="County Name", ignore_index=True)
data_CA_deaths = data_CA_deaths.iloc[:, 3:]
data_CA_deaths = data_CA_deaths.diff(axis = 1) #find the new daily deaths at each date

data_CA_cases = data_all_cases[data_all_cases['State'].values=='CA']
data_CA_cases.drop(data_CA_cases.loc[data_CA_cases['County Name']=="Statewide Unallocated"].index, inplace=True)
data_CA_cases.drop(data_CA_cases.loc[data_CA_cases['County Name']=="Grand Princess Cruise Ship"].index, inplace=True)
data_CA_cases = data_CA_cases.sort_values(by="County Name", ignore_index=True)
data_CA_cases = data_CA_cases.iloc[:, 3:]
data_CA_cases = data_CA_cases.diff(axis = 1) #find the new daily cases at each date

data_CA_deaths.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20
0,NaN,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,21.0,0.0,4.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,3.0,0.0,0.0,0.0,9.0,3.0,0.0,3.0,1.0,0.0,3.0,1.0
1,NaN,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,NaN,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Get the mask usage data from NY times
!git clone https://github.com/nytimes/covid-19-data

Cloning into 'covid-19-data'...
remote: Enumerating objects: 9817, done.
remote: Total 9817 (delta 0), reused 0 (delta 0), pack-reused 9817
Receiving objects: 100% (9817/9817), 486.53 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (6093/6093), done.


In [ ]:
#mask data
mask_use = pd.read_csv("/content/covid-19-data/mask-use/mask-use-by-county.csv")
mask_use = mask_use.sort_values(by="COUNTYFP", ignore_index=True)
mask_use.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [ ]:
#Get the FIPS code for each county
!git clone https://github.com/kjhealy/fips-codes

Cloning into 'fips-codes'...
remote: Enumerating objects: 13, done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (13/13), done.


In [ ]:
#FIPS codes
FIPS_codes = pd.read_csv("/content/fips-codes/county_fips_master.csv", encoding='latin-1')
FIPS_codes = FIPS_codes.sort_values(by="fips", ignore_index=True)
FIPS_codes.head()

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,1001,Autauga County,AL,Alabama,Autauga County AL,50.0,3.0,6.0,1.0,1.0,3-6-1-1,South,East South Central
1,1003,Baldwin County,AL,Alabama,Baldwin County AL,50.0,3.0,6.0,1.0,3.0,3-6-1-3,South,East South Central
2,1005,Barbour County,AL,Alabama,Barbour County AL,50.0,3.0,6.0,1.0,5.0,3-6-1-5,South,East South Central
3,1007,Bibb County,AL,Alabama,Bibb County AL,50.0,3.0,6.0,1.0,7.0,3-6-1-7,South,East South Central
4,1009,Blount County,AL,Alabama,Blount County AL,50.0,3.0,6.0,1.0,9.0,3-6-1-9,South,East South Central


In [ ]:
#Adding county FIPS to the mask usage data
state = 'CA'
state_indices = FIPS_codes[FIPS_codes["state_abbr"] == state].index.tolist()
first_fips_value = FIPS_codes.iloc[state_indices[0], 0]
last_fips_value = FIPS_codes.iloc[state_indices[-1], 0]

first_fips_index = mask_use[mask_use["COUNTYFP"] == first_fips_value].index.tolist()
last_fips_index = mask_use[mask_use["COUNTYFP"] == last_fips_value].index.tolist()
mask_use_CA = mask_use.iloc[first_fips_index[0]:last_fips_index[0] + 1 , :]

county_names = FIPS_codes.iloc[state_indices, 1].values
mask_use_CA["County_name"] = county_names
mask_use_CA = mask_use_CA.sort_values(by="County_name", ignore_index=True)
mask_use_CA.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,County_name
0,6001,0.019,0.008,0.055,0.123,0.795,Alameda County
1,6003,0.025,0.085,0.088,0.190,0.612,Alpine County
2,6005,0.045,0.013,0.099,0.188,0.655,Amador County
3,6007,0.015,0.043,0.111,0.204,0.626,Butte County
4,6009,0.045,0.019,0.098,0.276,0.562,Calaveras County


In [ ]:
#Load the area data
area = pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/Population_CA.csv")
area = area.sort_values(by="County", ignore_index=True)
area = area.drop(columns=['Population', 'Population density (person/km^2)'])
area = area.iloc[:, 0:3]
area.head()

,County,Area (sq mile),Area (sq km)
0,"Alameda, CA",739,1914.054410
1,"Alpine, CA",738,1912.267317
2,"Amador, CA",595,1539.956254
3,"Butte, CA","1,636",4238.415035
4,"Calaveras, CA","1,020",2641.815700


In [ ]:
#Load the high school grads data
high_school = pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/Educational Attainment Percent high school graduate or higher by County.csv")
high_school.drop(high_school.tail(1).index, inplace=True)  #drop the last row
high_school.drop(high_school.columns[-1], axis=1, inplace=True) #drop the last column (margin of error column)

new_header = high_school.iloc[0] #grab the first row for the header
high_school = high_school[1:] #take the data less the header row
high_school.columns = new_header
high_school = high_school.reset_index()
high_school = high_school.drop(columns=['index'])

high_school = high_school.sort_values(by="County", ignore_index=True)

high_school["Education"] = high_school["Education"].str.replace('%', '')
high_school["Education"] = high_school["Education"].astype(float)

high_school.head()

,County,Education
0,Alameda County,88.4
1,Alpine County,91.2
2,Amador County,90.0
3,Butte County,89.2
4,Calaveras County,90.2


In [ ]:
#Load the persons in poverty data
poverty = pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/Persons in poverty, percent by County.csv")
poverty.drop(poverty.tail(1).index, inplace=True)  #drop the last row

new_header = poverty.iloc[0] #grab the first row for the header
poverty = poverty[1:] #take the data less the header row
poverty.columns = new_header
poverty = poverty.reset_index()
poverty = poverty.drop(columns=['index'])

poverty = poverty.sort_values(by="County", ignore_index=True)

poverty["Individuals Below Poverty Level"] = poverty["Individuals Below Poverty Level"].str.replace('%', '')
poverty["Individuals Below Poverty Level"] = poverty["Individuals Below Poverty Level"].astype(float)

poverty.head()

,County,Individuals Below Poverty Level
0,Alameda County,8.9
1,Alpine County,17.2
2,Amador County,9.8
3,Butte County,16.1
4,Calaveras County,12.1


In [ ]:
#Load the persons per household data
persons = pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/Persons_per_household.csv")
persons = persons.iloc[:, 0:2]
persons = persons.sort_values(by="County", ignore_index=True)
persons.head()

,County,Value
0,Alameda,2.81
1,Alpine,3.71
2,Amador,2.42
3,Butte,2.55
4,Calaveras,2.57


In [ ]:
#Load the median income data
income = pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/Median Household Income by County.csv")
income.drop(income.tail(1).index, inplace=True)  #drop the last row
income.drop(income.columns[-1], axis=1, inplace=True) #drop the last column (margin of error column)

new_header = income.iloc[0] #grab the first row for the header
income = income[1:] #take the data less the header row
income.columns = new_header
income = income.reset_index()
income = income.drop(columns=['index'])

income = income.sort_values(by="County", ignore_index=True)

income["Income"] = income["Income"].str.replace('$', '')
income["Income"] = income["Income"].str.replace(',', '')
income["Income"] = income["Income"].astype(float)

income.head()

,County,Income
0,Alameda County,99406.0
1,Alpine County,63750.0
2,Amador County,62772.0
3,Butte County,52537.0
4,Calaveras County,63158.0


In [ ]:
#Add the election results and make it categorical
from pandas.api.types import CategoricalDtype
CA_election=pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/CA_election_results.csv", encoding= 'unicode_escape')
CA_election = CA_election.sort_values(by="COUNTY", ignore_index=True)

CA_election = CA_election.iloc[:, 1:]
CA_election.drop(CA_election.columns[0], axis=1, inplace=True)
CA_election.drop(CA_election.columns[1], axis=1, inplace=True)

CA_election.iloc[:, -1] = CA_election.iloc[:, -1].astype('str')
string_values = CA_election.iloc[:, -1].values
numerical_values = np.zeros(len(string_values))
for i in range(0, len(string_values)):
  temp = string_values[i].split('%')
  numerical_values[i] = float(temp[0])
CA_election.iloc[:, -1] = numerical_values

CA_election.iloc[:, 0] = CA_election.iloc[:, 0].astype('str')
string_values = CA_election.iloc[:, 0].values
numerical_values = np.zeros(len(string_values))
for i in range(0, len(string_values)):
  temp = string_values[i].split('%')
  numerical_values[i] = float(temp[0])
CA_election.iloc[:, 0] = numerical_values

CA_election['difference_in_votes'] = CA_election.iloc[:, 0] - CA_election.iloc[:, -1]
CA_election.loc[CA_election.difference_in_votes > 0, "difference_in_votes"] = 0
CA_election.loc[CA_election.difference_in_votes < 0, "difference_in_votes"] = 1
CA_election["difference_in_votes"] = CA_election["difference_in_votes"].astype('category')
CA_election["difference_in_votes"] = CA_election["difference_in_votes"].cat.codes
CA_election = pd.get_dummies(CA_election, columns=['difference_in_votes'])

os.chdir("/content/drive/My Drive/AI_Thermal sytems_course_project/California")
CA_election.to_csv('Election_CA_Miad.csv', index=False)

CA_election.head()

,BIDEN PCT,TRUMP PCT,difference_in_votes_0,difference_in_votes_1
0,80.2,17.7,1,0
1,64.7,32.6,1,0
2,36.6,60.9,0,1
3,49.4,47.8,1,0
4,36.8,61.0,0,1


In [ ]:
#Load the population data
data_CA_compact = pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/Population Estimates by County.csv")
data_CA_compact.drop(data_CA_compact.tail(1).index, inplace=True)  #drop the last row

new_header = data_CA_compact.iloc[0] #grab the first row for the header
data_CA_compact = data_CA_compact[1:] #take the data less the header row
data_CA_compact.columns = new_header
data_CA_compact = data_CA_compact.reset_index()
data_CA_compact = data_CA_compact.drop(columns=['index'])

data_CA_compact = data_CA_compact.sort_values(by="County", ignore_index=True)

data_CA_compact["Population"] = data_CA_compact["Population"].str.replace(',', '')
data_CA_compact["Population"] = data_CA_compact["Population"].astype(float)

##Add new columns
data_CA_compact["Area (sq km)"] = area["Area (sq km)"]
data_CA_compact["Population density (person/km^2)"] = data_CA_compact["Population"] / data_CA_compact["Area (sq km)"]
data_CA_compact["High school percentage"] = high_school["Education"]
data_CA_compact["Persons per household"] = persons["Value"]
data_CA_compact["Persons in poverty percentage"] = poverty["Individuals Below Poverty Level"]
data_CA_compact["Median income"] = income["Income"]
data_CA_compact['Biden won'] = CA_election['difference_in_votes_0']
data_CA_compact['Trump won'] = CA_election['difference_in_votes_1']
data_CA_compact['Never'] = mask_use_CA['NEVER']
data_CA_compact['Rarely'] = mask_use_CA['RARELY']
data_CA_compact['Sometimes'] = mask_use_CA['SOMETIMES']
data_CA_compact['Frequently'] = mask_use_CA['FREQUENTLY']
data_CA_compact['Always'] = mask_use_CA['ALWAYS']
data_CA_compact.head()

,County,Population,Area (sq km),Population density (person/km^2),High school percentage,Persons per household,Persons in poverty percentage,Median income,Biden won,Trump won,Never,Rarely,Sometimes,Frequently,Always
0,Alameda County,1671329.0,1914.054410,873.187821,88.4,2.81,8.9,99406.0,1,0,0.019,0.008,0.055,0.123,0.795
1,Alpine County,1129.0,1912.267317,0.590399,91.2,3.71,17.2,63750.0,1,0,0.025,0.085,0.088,0.190,0.612
2,Amador County,39752.0,1539.956254,25.813720,90.0,2.42,9.8,62772.0,0,1,0.045,0.013,0.099,0.188,0.655
3,Butte County,219186.0,4238.415035,51.714143,89.2,2.55,16.1,52537.0,1,0,0.015,0.043,0.111,0.204,0.626
4,Calaveras County,45905.0,2641.815700,17.376307,90.2,2.57,12.1,63158.0,0,1,0.045,0.019,0.098,0.276,0.562


In [ ]:
#Calculate the death ratios for all counties
CA_order_date = '6/18/20'

start = -1  #1 months before
end = 1 #1 months after
date_components = CA_order_date.split('/')
month = int(date_components[0])
day = int(date_components[1])
year = int(date_components[-1])

data_CA_compact = data_CA_compact.sort_values(by="County", ignore_index=True)

for i in range(start, end):    #add new columns of the average of cases and deaths in months before & after the order for all counties
  initial_date = str(month + i) + '/' + str(day) + '/' + str(year)
  end_date = str(month + i + 1) + '/' + str(day) + '/' + str(year)
  first_index =  data_CA_deaths.columns.get_loc(initial_date)
  last_index = data_CA_deaths.columns.get_loc(end_date)
  
  if i<0:
    data_CA_compact['%s_months_before_order deaths/cases percentage' %abs(i)] = data_CA_deaths.iloc[:,first_index:last_index].mean(axis=1).values / data_CA_cases.iloc[:,first_index:last_index].mean(axis=1).values*100
    column_index = data_CA_compact.columns.get_loc('%s_months_before_order deaths/cases percentage' %abs(i))
    rows_with_nan = []
    for index, row in data_CA_compact.iterrows():
      is_nan_series = row.isnull()
      if is_nan_series.any():
        rows_with_nan.append(index)
    for m in rows_with_nan:
      data_CA_compact.iloc[m, column_index] = data_CA_deaths.iloc[:,first_index:last_index].mean(axis=1).values[m]

  else:
    data_CA_compact['%s_months_after_order deaths/cases percentage' %(i+1)] = data_CA_deaths.iloc[:,first_index:last_index].mean(axis=1).values / data_CA_cases.iloc[:,first_index:last_index].mean(axis=1).values*100
    column_index = data_CA_compact.columns.get_loc('%s_months_after_order deaths/cases percentage' %(i+1))
    rows_with_nan = []
    for index, row in data_CA_compact.iterrows():
      is_nan_series = row.isnull()
      if is_nan_series.any():
        rows_with_nan.append(index)
    for m in rows_with_nan:
      data_CA_compact.iloc[m, column_index] = data_CA_deaths.iloc[:,first_index:last_index].mean(axis=1).values[m]

#Drop the rows containing a death ratio less than 0
data_CA_compact.drop(data_CA_compact.loc[data_CA_compact["1_months_before_order deaths/cases percentage"] < 0].index, inplace=True)
data_CA_compact.drop(data_CA_compact.loc[data_CA_compact["1_months_after_order deaths/cases percentage"] < 0].index, inplace=True)

import os
os.chdir("/content/drive/My Drive/AI_Thermal sytems_course_project/California")
data_CA_compact.to_csv('CA_death rates without NAN_copy.csv', index=False)  

data_CA_compact.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide


,County,Population,Area (sq km),Population density (person/km^2),High school percentage,Persons per household,Persons in poverty percentage,Median income,Biden won,Trump won,Never,Rarely,Sometimes,Frequently,Always,1_months_before_order deaths/cases percentage,1_months_after_order deaths/cases percentage
0,Alameda County,1671329.0,1914.054410,873.187821,88.4,2.81,8.9,99406.0,1,0,0.019,0.008,0.055,0.123,0.795,1.541336,1.040462
1,Alpine County,1129.0,1912.267317,0.590399,91.2,3.71,17.2,63750.0,1,0,0.025,0.085,0.088,0.190,0.612,0.000000,0.000000
2,Amador County,39752.0,1539.956254,25.813720,90.0,2.42,9.8,62772.0,0,1,0.045,0.013,0.099,0.188,0.655,0.000000,0.000000
3,Butte County,219186.0,4238.415035,51.714143,89.2,2.55,16.1,52537.0,1,0,0.015,0.043,0.111,0.204,0.626,1.470588,0.694444
4,Calaveras County,45905.0,2641.815700,17.376307,90.2,2.57,12.1,63158.0,0,1,0.045,0.019,0.098,0.276,0.562,0.000000,0.000000


In [ ]:
CA_all_death_rates_one_month_before_order = np.sum(data_CA_compact['1_months_before_order deaths/cases percentage'].values)
CA_all_death_rates_one_month_after_order = np.sum(data_CA_compact['1_months_after_order deaths/cases percentage'].values)
print ('total death rates 1 month before order:', CA_all_death_rates_one_month_before_order)
print ('total death rates 1 month after order:', CA_all_death_rates_one_month_after_order)
CA_difference_death_rates = data_CA_compact['1_months_after_order deaths/cases percentage'].values - data_CA_compact['1_months_before_order deaths/cases percentage'].values
reduction_count = np.argwhere(CA_difference_death_rates<0)
increase_count = np.argwhere(CA_difference_death_rates>0)
constant_count = np.argwhere(CA_difference_death_rates==0)
print ('number of reduction in death rates from one month before to one month after order:', len(reduction_count))
print ('number of increase in death rates from one month before to one month after order:', len(increase_count))
print ('number of no changes in death rates from one month before to one month after order:', len(constant_count))

total death rates 1 month before order: 63.13269173093065
total death rates 1 month after order: 32.669272603736744
number of reduction in death rates from one month before to one month after order: 28
number of increase in death rates from one month before to one month after order: 11
number of no changes in death rates from one month before to one month after order: 19


In [ ]:
#Add a last column to divided the change in death ratio into 3 categires of increase, decrease, and no change
data_CA_compact['change in deaths/cases percentage']=data_CA_compact['1_months_after_order deaths/cases percentage']-data_CA_compact['1_months_before_order deaths/cases percentage']
from pandas.api.types import CategoricalDtype # import module required to make a categorical data
data_CA_compact.loc[data_CA_compact['change in deaths/cases percentage'] > 0, "status of the change"] = "increase" #assign first category
data_CA_compact.loc[data_CA_compact['change in deaths/cases percentage'] < 0, "status of the change"] = "decrease" #assign second category
data_CA_compact.loc[data_CA_compact['change in deaths/cases percentage'] == 0, "status of the change"] = "no change" #assign third category
status_type = CategoricalDtype(categories=['increase', 'decrease','no change'], ordered=True) #define your items as categories
data_CA_compact['status of the change'] = data_CA_compact['status of the change'].astype(status_type) #change your data type to categories
print(data_CA_compact['status of the change'].dtype)# cheack for the data type

import os
os.chdir("/content/drive/My Drive/AI_Thermal sytems_course_project/California")
data_CA_compact.to_csv('Data_California_copy.csv', index=False) 

category


In [ ]:
os.chdir("/content/drive/My Drive/AI_Thermal sytems_course_project/California")
data_CA_compact.to_csv('Data_California_copy.csv', index=False) 
data_CA_compact.head()

,County,Population,Area (sq km),Population density (person/km^2),High school percentage,Persons per household,Persons in poverty percentage,Median income,Biden won,Trump won,Never,Rarely,Sometimes,Frequently,Always,1_months_before_order deaths/cases percentage,1_months_after_order deaths/cases percentage,change in deaths/cases percentage,status of the change
0,Alameda County,1671329.0,1914.054410,873.187821,88.4,2.81,8.9,99406.0,1,0,0.019,0.008,0.055,0.123,0.795,1.541336,1.040462,-0.500873,decrease
1,Alpine County,1129.0,1912.267317,0.590399,91.2,3.71,17.2,63750.0,1,0,0.025,0.085,0.088,0.190,0.612,0.000000,0.000000,0.000000,no change
2,Amador County,39752.0,1539.956254,25.813720,90.0,2.42,9.8,62772.0,0,1,0.045,0.013,0.099,0.188,0.655,0.000000,0.000000,0.000000,no change
3,Butte County,219186.0,4238.415035,51.714143,89.2,2.55,16.1,52537.0,1,0,0.015,0.043,0.111,0.204,0.626,1.470588,0.694444,-0.776144,decrease
4,Calaveras County,45905.0,2641.815700,17.376307,90.2,2.57,12.1,63158.0,0,1,0.045,0.019,0.098,0.276,0.562,0.000000,0.000000,0.000000,no change


In [ ]:
data_CA_compact.describe()

,Population,Area (sq km),Population density (person/km^2),High school percentage,Persons per household,Persons in poverty percentage,Median income,Biden won,Trump won,Never,Rarely,Sometimes,Frequently,Always,1_months_before_order deaths/cases percentage,1_months_after_order deaths/cases percentage,change in deaths/cases percentage
count,5.800000e+01,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.00000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000
mean,6.812452e+05,6956.320622,276.304422,84.687931,2.782414,13.020690,67713.603448,0.603448,0.396552,0.03231,0.030724,0.063690,0.158603,0.714638,1.088495,0.563263,-0.525231
std,1.467932e+06,8034.976492,976.491396,7.138138,0.400001,4.052539,20213.449856,0.493454,0.493454,0.02879,0.026426,0.034187,0.040760,0.093586,1.876019,0.603522,1.879101
min,1.129000e+03,121.392831,0.590399,69.100000,2.130000,6.100000,40846.000000,0.000000,0.000000,0.00100,0.000000,0.004000,0.058000,0.482000,0.000000,0.000000,-12.903226
25%,4.804825e+04,2485.063030,9.831872,80.025000,2.442500,9.875000,53430.500000,0.000000,0.000000,0.01525,0.013000,0.040000,0.134500,0.662750,0.000000,0.000000,-1.032213
50%,1.870290e+05,3976.515247,42.620728,86.500000,2.795000,12.650000,62516.000000,1.000000,0.000000,0.02350,0.023000,0.058500,0.156000,0.732500,0.557191,0.456935,0.000000
75%,7.092760e+05,8946.854981,139.764233,90.150000,3.065000,16.450000,80508.500000,1.000000,1.000000,0.04275,0.042750,0.084000,0.185250,0.786000,1.670379,0.923597,0.000000
max,1.003911e+07,51947.274030,7261.952708,95.100000,3.870000,22.000000,124055.000000,1.000000,1.000000,0.14000,0.135000,0.162000,0.276000,0.889000,12.903226,2.602705,1.997090
